## Command Line Training/Testing

### Setup ###
- install bazel
- clone tensorflow
- create image repo

### Retrain Inception ###
- navigate to tensorflow root
cd /home/jarmentr/.local/bin/175/tensorflow
- build retrainer
bazel build tensorflow/examples/image_retraining:retrain
- retrain the final layer of the inception model
bazel-bin/tensorflow/examples/image_retraining/retrain --image_dir /home/jarmentr/.local/bin/175/data/minecraft_screenshots  --output_graph /home/jarmentr/.local/bin/175/is_mc_graph.pb --output_labels /home/jarmentr/.local/bin/175/is_mc_labels.txt --bottleneck_dir /home/jarmentr/.local/bin/175/mc_bottleneck

###  Test Trained model ###
- bazel build tensorflow/examples/label_image:label_image
- bazel-bin/tensorflow/examples/label_image/label_image --graph=/home/jarmentr/.local/bin/175/is_mc_graph.pb --labels=/home/jarmentr/.local/bin/175/is_mc_labels.txt --output_layer=final_result --input_layer=Mul --image=/home/jarmentr/.local/bin/175/data/is_mc_test_01.jpg


In [1]:
import numpy as np
import tensorflow as tf

# retrained model
modelPath = '/home/jarmentr/.local/bin/175/is_mc_graph.pb'
labelsPath = '/home/jarmentr/.local/bin/175/is_mc_labels.txt'

In [2]:
def get_labels(labelsPath):
    with open(labelsPath, 'r') as f:
        return f.read().strip().split('\n')

In [3]:
def create_graph():
    with tf.gfile.FastGFile(modelPath, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

In [4]:
def run_inference_on_image(labels, imgPath):
    if not tf.gfile.Exists(imgPath):
        tf.logging.fatal('File does not exist {0}'.format(imgPath))
        return None
    
    img_data = tf.gfile.FastGFile(imgPath, 'rb').read()
    
    with tf.Session() as sess:
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        preds = sess.run(softmax_tensor, {'DecodeJpeg/contents:0': img_data})
        preds = np.squeeze(preds)
        
        #top_k_ids = preds.argsort()[::-1]
        top_k = [(labels[k_id], preds[k_id]) for k_id in range(len(labels))]
        
        for label, score in top_k:
            print('{0} (score={1:.5f})'.format(label, score))
            
        return top_k

In [5]:
labels = get_labels(labelsPath)
create_graph()

In [26]:
imgPath = 'data/is_mc_test_01.jpg'
run_inference_on_image(labels, imgPath)
print('---------------')
imgPath = 'data/not_mc_test_01.jpg'
is_mc, not_mc = run_inference_on_image(labels, imgPath)
print(is_mc)
print('---------------')

is minecraft (score=0.57628)
not minecraft (score=0.42372)
---------------
is minecraft (score=0.00062)
not minecraft (score=0.99938)
('is minecraft', 0.0006164423)
---------------


In [29]:
import os, shutil
test_img_dir   = 'data/minecraft_pig_GoogleSearch/'
is_mc_dir      = 'data/is_mc_dir/'
not_mc_dir     = 'data/not_mc_dir/'
invalid_mc_dir = 'data/invalid_mc_dir/'
is_mc_count, not_mc_count, invalid_mc_count = 0

with open('data/mc_pig_logs', 'w') as f:
    for path in os.listdir(test_base):
        try:
            is_mc, not_mc = run_inference_on_image(labels, test_img_dir + path)
            line = '{0},{1},{2}'.format(path, is_mc[1], not_mc[1])
            print(line)
            f.write(line + '\n')
            if is_mc[1] > 0.5:
                is_mc_count += 1
                shutil.copyfile(test_img_dir + path, is_mc_dir + path)
            else:
                not_mc_count += 1
                shutil.copyfile(test_img_dir + path, not_mc_dir + path)
        except Exception as e:
            # tensorflow model unable to parse image
            invalid_mc_count += 1
            shutil.move(test_img_dir + path, invalid_mc_dir)

is minecraft (score=0.04113)
not minecraft (score=0.95887)
pic_003.jpg,is minecraft:0.04112888500094414,not minecraft:0.958871066570282
is minecraft (score=0.00122)
not minecraft (score=0.99878)
pic_005.jpg,is minecraft:0.0012199269840493798,not minecraft:0.9987800717353821
is minecraft (score=0.00172)
not minecraft (score=0.99828)
pic_006.jpg,is minecraft:0.0017214925028383732,not minecraft:0.9982784986495972
is minecraft (score=0.00185)
not minecraft (score=0.99815)
pic_007.jpg,is minecraft:0.0018546307692304254,not minecraft:0.998145341873169
is minecraft (score=0.00837)
not minecraft (score=0.99163)
pic_008.jpg,is minecraft:0.00836517196148634,not minecraft:0.9916348457336426
is minecraft (score=0.15954)
not minecraft (score=0.84046)
pic_011.jpg,is minecraft:0.1595396250486374,not minecraft:0.8404603600502014
is minecraft (score=0.73631)
not minecraft (score=0.26369)
pic_012.jpg,is minecraft:0.7363088726997375,not minecraft:0.26369109749794006
is minecraft (score=0.66515)
not minec

In [32]:
print(is_mc_count)
print(not_mc_count)
print(invalid_mc_count)
print(is_mc_count + not_mc_count)

42
161
22
203


42
